#1.

#Parameters

$n$:numberof warehouses sources

$m$:number of market or demand points

$i$:denotes ith warehouse i=1,2,3,4.......n

$j$:donotes jth market j=1,2,3,4..........m

$dij$:Distance between warehouse i & market j

$fi$:fixed cost opening warehouse i=1,2,3....n

$vi$:variable cost for handling inventory(in 000s) in warehouse i=1,2....n

$Li,Ui$:minimum and maximum capacity of warehouse i=1,2,.....n

$Dj$:Annual demand(in 000s) for market j=1,2,3....n

$p1$:per(in 000s) transportation cost for upto 9000 units =6₹

$p2$:per(000s) trasportation cost more than 9000 units =4₹


#Decision Variable

$xi$=1 if warehouse is open,0 Otherwise

$yij$:Quantity shipped from warehouse i to retailer j

$y1ij$:Quantity shipped form warehouse i to market j less than breakpoint

$y2ij$:total quantity shipped form warehouse i to market j more than breakpoint

$zij$=1 if economy of scale achieved ,0 otherwise










#Formulation
\begin{align*}
\textbf{Minimize:} \quad & \sum_{i=1}^{n} f_ix_i + \sum_{i=1}^{n} \sum_{j=1}^{m} v_i y_{ij}\
+ \sum_{i=1}^{n} \sum_{j=1}^{m} p_{1}y_{1ij}d_{ij}+\sum_{i=1}^{n} \sum_{j=1}^{m} p_{2}y_{2ij}d_{ij}\\
\textbf{Constraints:}\\
\quad & \sum_{i=1}^{n}y_{ij}\geq D_j \quad \forall j = 1,2,3,\ldots,m \\
\quad & \sum_{j=1}^{m} y_{ij} \leq U_i x_i, \quad \forall i = 1,2,3,\ldots,n \\
\quad & \sum_{j=1}^{m} y_{ij} \geq L_i x_i, \quad \forall i = 1,2,3,\ldots,n \\
\quad & 0\leq y_{1ij} \leq 9000(1-z_{ij}), \quad \forall i = 1,2,3,\ldots,n, \quad \forall j = 1,2,3,\ldots,m \\
\quad & 9000z_{ij}\leq y_{2ij} \leq Mz_{ij} \quad \forall i = 1,2,3,\ldots,n, \quad \forall j = 1,2,3,\ldots,m \\
\quad & \sum_{j=1}^{m} y_{ij} = \sum_{j=1}^{m} y_{1ij} + \sum_{j=1}^{m} y_{2ij}, \quad \forall i = 1,2,3,\ldots,n \\
\quad & x_i \in \{0, 1\}, \quad \forall i = 1,2,3,\ldots,n \\
\quad & y_{ij} \geq 0, \quad \forall i = 1,2,3,\ldots,n, \quad \forall j = 1,2,3,\ldots,m \\
\quad & y_{1ij} \geq 0, \quad \forall i = 1,2,3,\ldots,n, \quad \forall j = 1,2,3,\ldots,m \\
\quad & y_{2ij} \geq 0, \quad \forall i = 1,2,3,\ldots,n, \quad \forall j = 1,2,3,\ldots,m \\
\quad & z_{ij} \in \{0, 1\}, \quad \forall i = 1,2,3,\ldots,n \\
\end{align*}


In [ ]:
!pip install pyomo

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from pyomo.environ import *

In [ ]:
model = ConcreteModel()

In [ ]:
model.m = Set(initialize = range(9))
model.n = Set(initialize = range(4))

In [ ]:
d_ij =  [[120,140,0,270,365,180,310,680,480],
              [340,515,365,320,0,175,290,660,120],
              [130,320,180,250,175,0,290,590,310],
              [400,420,310,30,290,290,0,370,210]]
model.L_i = Param(model.n,default = 20)
model.U_i = Param(model.n,default = 40)
model.D_j = Param(model.m,initialize = [12,8,14,16,7,15,17,8,9])
model.v_i = Param(model.n,initialize = [12000,10000,13000,13000])
model.f_i = Param(model.n,initialize = [ 550000,710000, 610000,650000])
model.p1 =  Param(default = 6)
model.p2 = Param(default = 4)
model.d = Param(model.n,model.m,default = 0,mutable = True)

In [ ]:
for i in model.n :
    for j in model.m:
        model.d[i,j] = d_ij[i][j]

In [ ]:
model.x_i = Var(model.n,within = Binary)
model.y_ij = Var(model.n,model.m,within = NonNegativeIntegers)
model.y1_ij = Var(model.n,model.m,within = NonNegativeIntegers)
model.y2_ij = Var(model.n,model.m,within = NonNegativeIntegers)
model.z_ij = Var(model.n,model.m,within = Binary)

In [ ]:
O1=sum(model.f_i[i]*model.x_i[i] for i in model.n)
O2=sum(model.v_i[i]*model.y_ij[i,j] for i in model.n for j in model.m)
O3=sum(model.p1*model.y1_ij[i,j]*model.d[i,j] for i in model.n for j in model.m)
O4=sum(model.p2*model.y2_ij[i,j]*model.d[i,j] for i in model.n for j in model.m)

In [ ]:
model.obj = Objective(expr = O1+O2+O3+O4,sense=minimize)

In [ ]:
model.constraints = ConstraintList()

In [ ]:
for j in model.m:
  model.constraints.add(expr = sum(model.y_ij[i,j] for i in model.n) >= model.D_j[j] )

In [ ]:
for i in model.n:
  model.constraints.add(expr = sum(model.y_ij[i,j] for j in model.m) >= (model.L_i[i] * model.x_i[i]) )

In [ ]:
for i in model.n:
  model.constraints.add(expr = sum(model.y_ij[i,j] for j in model.m) <= (model.U_i[i] * model.x_i[i]) )

In [ ]:
for i in model.n:
  for j in model.m:
    model.constraints.add(expr = model.y1_ij[i,j] <= (9 * (1 - model.z_ij[i,j])))

In [ ]:
for i in model.n:
  for j in model.m:
    model.constraints.add(expr = model.y2_ij[i,j] >= (9* model.z_ij[i,j]))

In [ ]:
for i in model.n:
  for j in model.m:
    model.constraints.add(expr = model.y2_ij[i,j] <= (10000000000000000000* model.z_ij[i,j]))

In [ ]:
for i in model.n:
  for j in model.m:
    model.constraints.add(expr = model.y_ij[i,j] ==  model.y1_ij[i,j]+ model.y2_ij[i,j])

In [ ]:
model.pprint()

8 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  161 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161}
    d_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    n*m :   36 : {(

In [ ]:
opt = SolverFactory('cplex')
result = opt.solve(model)
print(result)
print('Solver Status:',result.solver.status)
print('Solver time:',result.solver.time)
print('Solver termination condition: ',result.solver.termination_condition)


Problem: 
- Name: tmppyt_ovnu
  Lower bound: 3173780.0
  Upper bound: 3173940.0
  Number of objectives: 1
  Number of constraints: 161
  Number of variables: 148
  Number of nonzeros: 440
  Sense: minimize
Solver: 
- Status: ok
  User time: 0.07
  Termination condition: optimal
  Termination message: MIP - Integer optimal, tolerance (0.0001/1e-06)\x3a Objective = 3.1739400000e+06
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08270978927612305
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver Status: ok
Solver time: 0.08270978927612305
Solver termination condition:  optimal


In [ ]:
print("objective value ",model.obj())
print("____________________________________________________________________________________________")
for i in model.n :
    print(f"X[{i}] ",model.x_i[i].value)
print("_______________________________________________________________________________________________")

for i in model.n:
    for j in model.m:
        print(f'y[{i,j}]',model.y_ij[i,j].value)

objective value  3173940.0
____________________________________________________________________________________________
X[0]  1.0
X[1]  1.0
X[2]  1.0
X[3]  0.0
_______________________________________________________________________________________________
y[(0, 0)] 2.0
y[(0, 1)] 8.0
y[(0, 2)] 14.0
y[(0, 3)] 16.0
y[(0, 4)] -0.0
y[(0, 5)] -0.0
y[(0, 6)] -0.0
y[(0, 7)] 0.0
y[(0, 8)] -0.0
y[(1, 0)] -0.0
y[(1, 1)] -0.0
y[(1, 2)] -0.0
y[(1, 3)] -0.0
y[(1, 4)] 7.0
y[(1, 5)] -0.0
y[(1, 6)] 17.0
y[(1, 7)] 7.0
y[(1, 8)] 9.0
y[(2, 0)] 10.0
y[(2, 1)] -0.0
y[(2, 2)] -0.0
y[(2, 3)] -0.0
y[(2, 4)] -0.0
y[(2, 5)] 15.0
y[(2, 6)] -0.0
y[(2, 7)] 1.0
y[(2, 8)] -0.0
y[(3, 0)] -0.0
y[(3, 1)] -0.0
y[(3, 2)] -0.0
y[(3, 3)] -0.0
y[(3, 4)] -0.0
y[(3, 5)] -0.0
y[(3, 6)] -0.0
y[(3, 7)] -0.0
y[(3, 8)] -0.0


#2.

In [ ]:
for i in model.n:
    for j in model.m:
        model.y_ij[i, j].domain = NonNegativeReals
        model.y1_ij[i, j].domain = NonNegativeReals
        model.y2_ij[i, j].domain = NonNegativeReals

In [ ]:
opt = SolverFactory('cplex')
result = opt.solve(model)
print(result)
print('Solver Status:',result.solver.status)
print('Solver time:',result.solver.time)
print('Solver termination condition: ',result.solver.termination_condition)


Problem: 
- Name: tmp782mo3we
  Lower bound: 3173940.0
  Upper bound: 3173940.0
  Number of objectives: 1
  Number of constraints: 161
  Number of variables: 148
  Number of nonzeros: 440
  Sense: minimize
Solver: 
- Status: ok
  User time: 0.03
  Termination condition: optimal
  Termination message: MIP - Integer optimal solution\x3a Objective = 3.1739400000e+06
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04774975776672363
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver Status: ok
Solver time: 0.04774975776672363
Solver termination condition:  optimal


#4.
\begin{align*}
\sum_{i \in n} x_{ij} = 1, \quad \forall j \in m\\
 \quad y_{ij} \geq D_j \cdot x_{ij} \forall i \in n, \forall j \in m:
\end{align*}
where xij: 1 if  ith warehouse is open to shiping jth market ,0 Otherwise

In [ ]:
for i in model.n:
    for j in model.m:
        model.y_ij[i, j].domain = NonNegativeIntegers
        model.y1_ij[i, j].domain = NonNegativeIntegers
        model.y2_ij[i, j].domain = NonNegativeIntegers

In [ ]:
model.x_ij = Var(model.n,model.m,within = Binary)

In [ ]:
for i in model.n:
    for j in model.m:
        model.constraints.add(expr = model.y_ij[i,j]>=model.D_j[j]*model.x_ij[i,j])
        print(model.constraints.add(expr = model.y_ij[i,j]>=model.D_j[j]*model.x_ij[i,j]))

constraints[163]
constraints[165]
constraints[167]
constraints[169]
constraints[171]
constraints[173]
constraints[175]
constraints[177]
constraints[179]
constraints[181]
constraints[183]
constraints[185]
constraints[187]
constraints[189]
constraints[191]
constraints[193]
constraints[195]
constraints[197]
constraints[199]
constraints[201]
constraints[203]
constraints[205]
constraints[207]
constraints[209]
constraints[211]
constraints[213]
constraints[215]
constraints[217]
constraints[219]
constraints[221]
constraints[223]
constraints[225]
constraints[227]
constraints[229]
constraints[231]
constraints[233]


In [ ]:
for j in model.m:
    model.constraints.add(expr =sum(model.x_ij[i,j] for i in model.n)==1)
    print(model.constraints.add(expr =sum(model.x_ij[i,j] for i in model.n)==1) )

constraints[235]
constraints[237]
constraints[239]
constraints[241]
constraints[243]
constraints[245]
constraints[247]
constraints[249]
constraints[251]


In [ ]:
opt= SolverFactory('cplex')
result = opt.solve(model)
print(result1)
print('Solver Status:',result.solver.status)
print('Solver time:',result.solver.time)
print('Solver termination condition: ',result.solver.termination_condition)


Problem: 
- Name: tmp6wdgddjf
  Lower bound: 3179320.0
  Upper bound: 3179520.0
  Number of objectives: 1
  Number of constraints: 251
  Number of variables: 184
  Number of nonzeros: 656
  Sense: minimize
Solver: 
- Status: ok
  User time: 0.07
  Termination condition: optimal
  Termination message: MIP - Integer optimal, tolerance (0.0001/1e-06)\x3a Objective = 3.1795200000e+06
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08730411529541016
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver Status: ok
Solver time: 0.06911993026733398
Solver termination condition:  optimal


In [ ]:
print("objective value ",model.obj())
print("____________________________________________________________________________________________")
for i in model.n :
    print(f"X[{i}] ",model.x_i[i].value)
print("_______________________________________________________________________________________________")

for i in model.n:
    for j in model.m:
        if model.y_ij[i,j].value != 0:
            print(f'y[{i,j}]',model.y_ij[i,j].value)

objective value  3179520.0
____________________________________________________________________________________________
X[0]  1.0
X[1]  1.0
X[2]  1.0
X[3]  0.0
_______________________________________________________________________________________________
y[(0, 1)] 8.0
y[(0, 2)] 14.0
y[(0, 6)] 17.0
y[(1, 3)] 16.0
y[(1, 4)] 7.0
y[(1, 7)] 8.0
y[(1, 8)] 9.0
y[(2, 0)] 12.0
y[(2, 5)] 15.0


In [ ]:
for i in range(163,252,2):
    model.constraints[i].deactivate()

#5.

In [ ]:
model.L1_i = Param(model.n,default = 0)
model.U1_i = Param(model.n,default = 25)

In [ ]:
for i in model.n:
  model.constraints.add(expr = sum(model.y_ij[i,j] for j in model.m) >= (model.L1_i[i] * model.x_i[i]))

In [ ]:
for i in model.n:
  model.constraints.add(expr = sum(model.y_ij[i,j] for j in model.m) <= (model.U1_i[i] * model.x_i[i]) )

In [ ]:
opt = SolverFactory('cplex')
result = opt.solve(model)
print(result)
print('Solver Status:',result.solver.status)
print('Solver time:',result.solver.time)
print('Solver termination condition: ',result.solver.termination_condition)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 214
  Number of variables: 184
  Number of nonzeros: 624
  Sense: unknown
Solver: 
- Status: ok
  User time: 0.01
  Termination condition: infeasible
  Termination message: MIP - Integer infeasible.
  Error rc: 0
  Time: 0.020827054977416992

Solver Status: ok
Solver time: 0.020827054977416992
Solver termination condition:  infeasible
